In [ ]:
import pandas as pd
from sklearn.metrics import roc_curve, roc_auc_score, recall_score
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import seaborn as sns
sns.set_theme()
import warnings
warnings.filterwarnings("ignore")

In [ ]:
dataset_paths = ['', '']
cystic_artery_velocity_negative_cases_path = ''

# "Easy negatives" are patients who were determined not to have acute cholecystitis based on clinical grounds, and did not undergo surgery
include_easy_negatives = True

# "Hard negatives" are patients who underwent cholecystectomy but pathology confirmed no acute cholecystitis.
include_hard_negatives = False
include_chronic_cholecystitis = False

# value to use for 'equivocal' categorical feature interpretation
# None: equivocal cases are excluded
# 1: equivocal feature interpretations are treated as 'present'
# 0: equivocal feature interpretations are treated as 'absent'
surrogate_for_equivocal = None

demographics_paths = [('', ''),
                       ('', '')]

# continuous variables
parameter_columns = [
    'API_US_cystic_artery_velocity_max(cm/s)',
    'API_US_hepatic_artery_velocity_max(cm/s)',
    'API_US_GB_wall_thickness_max (mm)',
    'API_US_gallbladder_size(cm)_dimension1',
    'API_US_gallbladder_size(cm)_dimension2',
    'gallbladder_area'
                    ]

# categorical variables
categorical_parameter_columns = [
    'API_US_gallstone_cleaned',
    'API_US_gallbladder_sludge_cleaned',
    'API_US_GB_wall_thick (yes/no)_cleaned',
    'API_US_pericholecystic_fluid_cleaned',
    'API_US_sonographic_murphys_cleaned',
                                ]
                                 
outcome_columns = ['patho_acute_cholecystitis']

demographics_columns = [
    'Age_rad',
    'Gender',
    'Race',
    'Ethnicity',
    'Recent BMI'
]

set_prevalence = 0.5  # None if use the prevalence in the dataset

export_dir = ''

np.random.seed(0)

if include_chronic_cholecystitis:
    map_col_to_selected_thresholds = {
        'API_US_cystic_artery_velocity_max(cm/s)': [42.6, 24.6, 34.0],
        'API_US_hepatic_artery_velocity_max(cm/s)': [95.5],
        'API_US_GB_wall_thickness_max (mm)': [4.0],
        'API_US_gallbladder_size(cm)_dimension1': [8.4],
        'API_US_gallbladder_size(cm)_dimension2': [3.8, 2.7, 2.2],
        'gallbladder_area': [30.36],
    }

if include_easy_negatives:
    map_col_to_selected_thresholds = {
        'API_US_cystic_artery_velocity_max(cm/s)': [42.6, 24.6, 34.0],
        'API_US_hepatic_artery_velocity_max(cm/s)': [95.5],
        'API_US_GB_wall_thickness_max (mm)': [4.0],
        'API_US_gallbladder_size(cm)_dimension1': [8.4],
        'API_US_gallbladder_size(cm)_dimension2': [3.8, 2.7, 2.2],
        'gallbladder_area': [30.36, 39.60, 17.36],
    }

map_col_to_units = {
    'API_US_cystic_artery_velocity_max(cm/s)': 'cm/s',
    'API_US_hepatic_artery_velocity_max(cm/s)': 'cm/s',
    'API_US_GB_wall_thickness_max (mm)': 'mm',
    'API_US_gallbladder_size(cm)_dimension1': 'cm',
    'API_US_gallbladder_size(cm)_dimension2': 'cm',
    'gallbladder_area': 'cm\u00B2'
}

In [ ]:
def read_excel_add_add_source(path):
    df = pd.read_excel(path)
    df['source'] = path
    return df

##############################################

df = pd.concat([read_excel_add_add_source(path) for path in dataset_paths], 
               axis=0, 
               ignore_index=True)
print(df.shape)

df.loc[(df['patho_acute_cholecystitis']=='yes') | (df['cholecystostomy_acute_cholecystitis']=='yes'), 'comparison_group'] = 'positives'
df.loc[(df['patho_acute_cholecystitis']=='no') | (df['cholecystostomy_acute_cholecystitis']=='no'), 'comparison_group'] = 'hard negatives'

if not include_hard_negatives:
    df = df.loc[(df['patho_acute_cholecystitis']=='yes') | (df['cholecystostomy_acute_cholecystitis']=='yes')].reset_index(drop=True)
elif include_hard_negatives:
    if include_chronic_cholecystitis:
        df = df.loc[(df['patho_acute_cholecystitis']=='yes') | \
                    ((df['patho_acute_cholecystitis']=='no') & (df['patho_chronic_cholecystitis']=='yes'))].reset_index(drop=True)

if (cystic_artery_velocity_negative_cases_path is not None) and include_easy_negatives:
    cystic_df = read_excel_add_add_source(cystic_artery_velocity_negative_cases_path)

    cystic_df = cystic_df.loc[(cystic_df['US_acute_cholecystitis'].isin(['no','dirty negative'])) & \
                            (cystic_df['priority']==7)].reset_index(drop=True)
    
    # set these columns to 'no', so that the cases are included as negative controls
    cystic_df['patho_acute_cholecystitis'] = 'no'
    cystic_df['patho_gangrenous'] = 'no'
    cystic_df['cholecystostomy_acute_cholecystitis'] = 'no'

    cystic_df['comparison_group'] = 'easy negatives'

    df = pd.concat([df, cystic_df],
                  axis=0,
                  ignore_index=True)

print(df.shape)
print(df.columns)

# add demographics

In [ ]:
def read_demographics(demographics_path, codebook_path):
    demographics = pd.read_csv(demographics_path)
    codebook = pd.read_csv(codebook_path)
    codebook = codebook.rename(columns={'Patient id':'Patient Id'})
    demographics = demographics[['Patient Id',
                                 'Gender',
                                 'Race',
                                 'Ethnicity',
                                 'Recent Height cm',
                                 'Recent Weight kg',
                                 'Recent BMI']].merge(codebook[['Patient Id','MRN']], 
                                                    on='Patient Id', 
                                                    how='inner')
    return demographics

In [ ]:
print(df.shape)
demographics_df = pd.concat([read_demographics(demographics_path, codebook_path) for demographics_path, codebook_path in demographics_paths],
                            axis=0,
                            ignore_index=True)

demographics_df = demographics_df.drop_duplicates(subset='MRN').reset_index(drop=True)

df = df.merge(demographics_df,
                on='MRN',
                how='left').reset_index(drop=True)

print(df.shape)

In [ ]:
# do not use gallbladder sizes from cases where only one dimension is available

dimension_columns = ['API_US_gallbladder_size(cm)_dimension1',
                    'API_US_gallbladder_size(cm)_dimension2',
                    'API_US_gallbladder_size(cm)_dimension3']

indices_to_delete = df['API_US_gallbladder_size(cm)_dimension1'].isna() | df['API_US_gallbladder_size(cm)_dimension2'].isna()
for col in dimension_columns:
    df.loc[indices_to_delete, col] = None


# calculate gallbladder area

In [ ]:
available_indices = df['API_US_gallbladder_size(cm)_dimension1'].notna() & df['API_US_gallbladder_size(cm)_dimension2'].notna()
df.loc[available_indices, 'gallbladder_area'] = df.loc[available_indices]['API_US_gallbladder_size(cm)_dimension1'] * df.loc[available_indices]['API_US_gallbladder_size(cm)_dimension2']


# Descriptives

In [ ]:
def descriptives(df, outcome_col):
    print(outcome_col)
    sub_df = df.loc[df[outcome_col].notna()].copy().reset_index(drop=True)
    for group in ['positives', 'hard negatives', 'easy negatives']:
        print(group)
        group_df = sub_df.loc[sub_df['comparison_group']==group].copy().reset_index(drop=True)
        print('number of cases:', group_df.shape[0])
        for col in demographics_columns:
            print(group_df[col].value_counts(dropna=False))
            print(group_df[col].value_counts(normalize=True, dropna=False))
            print(group_df[col].describe())
            print('\n')
        for col in parameter_columns:
            print(group_df[col].describe())
            print('\n')
        for col in categorical_parameter_columns:
            print(group_df[col].value_counts(dropna=False))
            print(group_df[col].value_counts(normalize=True, dropna=False))
            print('\n')
        for col in outcome_columns:
            print(group_df[col].value_counts(dropna=False))
            print(group_df[col].value_counts(normalize=True, dropna=False))
            print('\n')

        

In [ ]:
for outcome_col in outcome_columns:
    descriptives(df, outcome_col)

# Begin analyses

In [ ]:
df['patho_acute_cholecystitis'].value_counts()

In [ ]:
df.loc[df['patho_acute_cholecystitis']=='no']['patho_chronic_cholecystitis'].value_counts(dropna=False)

In [ ]:
df.loc[~(df['priority'].isin([4,6])), 'final_Dx_acute_cholecystitis'] = df.loc[~(df['priority'].isin([4,6]))]['patho_acute_cholecystitis']
df.loc[df['priority'].isin([4,6]), 'final_Dx_acute_cholecystitis'] = df.loc[df['priority'].isin([4,6])]['cholecystostomy_acute_cholecystitis']


In [ ]:
df['final_Dx_acute_cholecystitis'].value_counts()

In [ ]:
for row_idx in df.index:
    acute = df.loc[row_idx, 'patho_acute_cholecystitis']
    if acute == 'yes':
        gangrenous = df.loc[row_idx, 'patho_gangrenous']
        if gangrenous == 'yes':
            df.loc[row_idx, 'gangrenous_vs_nongangrenous_acute_cholecystitis'] = 'yes'
            df.loc[row_idx, 'gangrenous_vs_nongangrenous_acute_cholecystitis (assume cholecystostomy cases are gangrenous)'] = 'yes'
        elif gangrenous == 'no':
            df.loc[row_idx, 'gangrenous_vs_nongangrenous_acute_cholecystitis'] = 'no'
            df.loc[row_idx, 'gangrenous_vs_nongangrenous_acute_cholecystitis (assume cholecystostomy cases are gangrenous)'] = 'no'

    if df.loc[row_idx, 'cholecystostomy_acute_cholecystitis'] == 'yes':
        df.loc[row_idx, 'gangrenous_vs_nongangrenous_acute_cholecystitis (assume cholecystostomy cases are gangrenous)'] = 'yes'

In [ ]:
for parameter_col in parameter_columns + categorical_parameter_columns:
    print('\n')
    print(parameter_col)
    for outcome_col in outcome_columns:
        print(df.loc[df[parameter_col].notna()][outcome_col].value_counts())

# Diagnostic performance

In [ ]:
def evaluate_diagnostic_performance(df, parameter_col, outcome_col):

    sub_df = df.loc[df[parameter_col].notna() & \
                    df[outcome_col].notna()].copy().reset_index(drop=True)
    print(sub_df.shape)

    if sub_df.shape[0] == 0:
        return

    print(sub_df[outcome_col].value_counts())

    if set_prevalence is None:
        prevalence = (sub_df[outcome_col]=='yes').sum() / sub_df.shape[0]
    else:
        prevalence = set_prevalence
        
    fpr, tpr, thresholds = roc_curve(y_true = sub_df[outcome_col], 
                                     y_score = sub_df[parameter_col],
                                     pos_label = 'yes')
    
    auroc = roc_auc_score(y_true = sub_df[outcome_col]=='yes', 
                         y_score = sub_df[parameter_col])

    map_colname = {
        'API_US_cystic_artery_velocity_max(cm/s)': 'Cystic artery velocity',
        'API_US_hepatic_artery_velocity_max(cm/s)': 'Hepatic artery velocity',
        'API_US_GB_wall_thickness_max (mm)': 'Gallbladder wall thickness',
        'API_US_gallbladder_size(cm)_dimension1': 'Gallbladder longitudinal dimension',
        'API_US_gallbladder_size(cm)_dimension2': 'Gallbladder transverse dimension',
        'gallbladder_area': 'Gallbladder longitudinal * transverse'
    }

    fig = plt.figure(figsize=(6,6))
    ax = fig.add_subplot(111)
    ax.plot(fpr, tpr, label = f'AUROC = {auroc:.2f}')
    plt.xlabel('1 - specificity')
    plt.ylabel('sensitivity')
    plt.title(map_colname[parameter_col])
    plt.legend()

    # annotate thresholds on the plot
    for x,y,velocity in zip(fpr,tpr,thresholds):
        if round(velocity,2) in map_col_to_selected_thresholds[parameter_col]:
            ax.annotate('{0:.1f} {1}'.format(round(velocity,1), map_col_to_units[parameter_col]), 
                        xy=(x+0.02,y-0.04),
                        textcoords='offset points')
            ax.scatter(x, y, color='green')
        
    plt.show()

    # calculate other metrics
    sens = tpr
    spec = 1 - fpr
    ppv = (sens*prevalence) / ((sens*prevalence) + ((1-spec)*(1-prevalence)))
    npv = (spec*(1-prevalence)) / (((1-sens)*prevalence)+(spec*(1-prevalence)))
    pos_lr = sens / (1-spec)
    neg_lr = (1-sens) / spec
    
    # write results to dataframe
    out_df = pd.DataFrame({
        'prevalence': [prevalence]*len(thresholds),
        'threshold': thresholds,
        'sensitivity': sens,
        'specificity': spec,
        'PPV': ppv,
        'NPV': npv,
        'positive_likelihood_ratio': pos_lr,
        'negative_likelihood_ratio': neg_lr,
        'AUROC': [auroc]*len(thresholds)
    })

    out_df.to_excel(export_dir + parameter_col.replace('/','per') + ' prevalence_{0:.2f}'.format(prevalence) + '.xlsx',
                   index=False)

    # plot sens, spec, PPV, NPV for each threshold
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.plot(thresholds, sens, label = 'sensitivity')
    ax.plot(thresholds, spec, label = 'specificity')
    ax.plot(thresholds, ppv, label = 'PPV')
    ax.plot(thresholds, npv, label = 'NPV')
    plt.xlabel('thresholds')
    plt.ylabel('metrics')
    plt.title(parameter_col + '\n' + outcome_col + '\n' + 'prevalence {0:.2f}'.format(prevalence))
    plt.legend()
    plt.show()

    # plot LR+ and LR- for each threshold
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.plot(thresholds, pos_lr, label = 'LR+')
    ax.plot(thresholds, neg_lr, label = 'LR-')
    plt.xlabel('thresholds')
    plt.ylabel('metrics')
    plt.title(parameter_col + '\n' + outcome_col + '\n' + 'prevalence {0:.2f}'.format(prevalence))
    plt.legend()
    plt.show()

    print('N:', sub_df.shape[0])
    print('pretest probability:', prevalence)
    print('AUROC:', auroc)

    # find performance at 90% spec
    spec90_df = out_df.loc[out_df['specificity']>=0.9].reset_index(drop=True)
    spec90_df = spec90_df.sort_values(by=['sensitivity'], ascending=False, ignore_index=True)
    spec90_series = spec90_df.iloc[0,:]
    print('\nperformance at 0.9 specificity')
    print(spec90_series)

    # find performance at 90% sens
    sens90_df = out_df.loc[out_df['sensitivity']>=0.9].reset_index(drop=True)
    sens90_df = sens90_df.sort_values(by=['specificity'], ascending=False, ignore_index=True)
    sens90_series = sens90_df.iloc[0,:]
    print('\nperformance at 0.9 sensitivity')
    print(sens90_series)
    

In [ ]:
def evaluate_diagnostic_performance_categorical(df, parameter_col, outcome_col):

    df_copy = df.copy()

    df_copy.loc[:,parameter_col] = df_copy[parameter_col].map({'yes': 1,
                                                    'no': 0,
                                                    'equivocal': surrogate_for_equivocal})
    df_copy.loc[:,outcome_col] = df_copy[outcome_col].map({'yes': 1,
                                                'no': 0})

    sub_df = df_copy.loc[df_copy[parameter_col].notna() & \
                        df_copy[outcome_col].notna()].copy().reset_index(drop=True)
    print(sub_df.shape)

    if sub_df.shape[0] == 0:
        return

    print(sub_df[outcome_col].value_counts())

    if set_prevalence is None:
        prevalence = (sub_df[outcome_col]==1).sum() / sub_df.shape[0]
    else:
        prevalence = set_prevalence

    sens = recall_score(sub_df[outcome_col].astype(int),
                        sub_df[parameter_col].astype(int),
                        pos_label = 1)
    spec = recall_score(sub_df[outcome_col].astype(int),
                        sub_df[parameter_col].astype(int),
                        pos_label = 0)

    # calculate other metrics
    ppv = (sens*prevalence) / ((sens*prevalence) + ((1-spec)*(1-prevalence)))
    npv = (spec*(1-prevalence)) / (((1-sens)*prevalence)+(spec*(1-prevalence)))
    pos_lr = sens / (1-spec)
    neg_lr = (1-sens) / spec

    print('N:', sub_df.shape[0])
    print('pretest probability:', prevalence)
    print('sensitivity:', sens)
    print('specificity:', spec)
    print('LR+:', pos_lr)
    print('LR-:', neg_lr)
    print('PPV:', ppv)
    print('NPV:', npv)

In [ ]:
# bootstrap
def evaluate_diagnostic_performance_categorical_bootstrap(df, parameter_col, outcome_col):
    
    df_copy = df.copy()

    df_copy.loc[:,parameter_col] = df_copy[parameter_col].map({'yes': 1,
                                                    'no': 0,
                                                    'equivocal': surrogate_for_equivocal})
    df_copy.loc[:,outcome_col] = df_copy[outcome_col].map({'yes': 1,
                                                'no': 0})

    sub_df = df_copy.loc[df_copy[parameter_col].notna() & \
                        df_copy[outcome_col].notna()].copy().reset_index(drop=True)

    if sub_df.shape[0] == 0:
        return

    if set_prevalence is None:
        prevalence = (sub_df[outcome_col]==1).sum() / sub_df.shape[0]
    else:
        prevalence = set_prevalence

    sens = recall_score(sub_df[outcome_col].astype(int),
                        sub_df[parameter_col].astype(int),
                        pos_label = 1)
    spec = recall_score(sub_df[outcome_col].astype(int),
                        sub_df[parameter_col].astype(int),
                        pos_label = 0)

    # calculate other metrics
    ppv = (sens*prevalence) / ((sens*prevalence) + ((1-spec)*(1-prevalence)))
    npv = (spec*(1-prevalence)) / (((1-sens)*prevalence)+(spec*(1-prevalence)))
    pos_lr = sens / (1-spec)
    neg_lr = (1-sens) / spec

    # bootstrap
    n_bootstrap = 10000
    sens_bootstrap = np.zeros(n_bootstrap)
    spec_bootstrap = np.zeros(n_bootstrap)
    ppv_bootstrap = np.zeros(n_bootstrap)
    npv_bootstrap = np.zeros(n_bootstrap)
    pos_lr_bootstrap = np.zeros(n_bootstrap)
    neg_lr_bootstrap = np.zeros(n_bootstrap)

    for i in tqdm(range(n_bootstrap)):
        indices = np.random.choice(sub_df.index, size=sub_df.shape[0], replace=True)
        sens_bootstrap[i] = recall_score(sub_df.loc[indices][outcome_col].astype(int),
                                         sub_df.loc[indices][parameter_col].astype(int),
                                         pos_label = 1)
        spec_bootstrap[i] = recall_score(sub_df.loc[indices][outcome_col].astype(int),
                                         sub_df.loc[indices][parameter_col].astype(int),
                                         pos_label = 0)
        ppv_bootstrap[i] = (sens_bootstrap[i]*prevalence) / ((sens_bootstrap[i]*prevalence) + ((1-spec_bootstrap[i])*(1-prevalence)))
        npv_bootstrap[i] = (spec_bootstrap[i]*(1-prevalence)) / (((1-sens_bootstrap[i])*prevalence)+(spec_bootstrap[i]*(1-prevalence)))
        pos_lr_bootstrap[i] = sens_bootstrap[i] / (1-spec_bootstrap[i])
        neg_lr_bootstrap[i] = (1-sens_bootstrap[i]) / spec_bootstrap[i]

    # print bootstrap results
    print('sensitivity:', round(sens*100,1), '95% CI:', tuple(np.round(np.percentile(sens_bootstrap, [2.5, 97.5])*100,1)))
    print('specificity:', round(spec*100,1), '95% CI:', tuple(np.round(np.percentile(spec_bootstrap, [2.5, 97.5])*100,1)))
    print('LR+:', round(pos_lr,2), '95% CI:', tuple(np.round(np.percentile(pos_lr_bootstrap, [2.5, 97.5]),2)))
    print('LR-:', round(neg_lr,2), '95% CI:', tuple(np.round(np.percentile(neg_lr_bootstrap, [2.5, 97.5]),2)))
    print('PPV:', round(ppv*100,1), '95% CI:', tuple(np.round(np.percentile(ppv_bootstrap, [2.5, 97.5])*100,1)))
    print('NPV:', round(npv*100,1), '95% CI:', tuple(np.round(np.percentile(npv_bootstrap, [2.5, 97.5])*100,1)))
    
    

In [ ]:

# continuous variables
for parameter_col in parameter_columns:
    print('\n')
    print(parameter_col)
    for outcome_col in outcome_columns:
        print(outcome_col)
        evaluate_diagnostic_performance(df, parameter_col, outcome_col)

# categorical variables
for parameter_col in categorical_parameter_columns:
    print('\n')
    print(parameter_col)
    for outcome_col in outcome_columns:
        print(outcome_col)
        evaluate_diagnostic_performance_categorical(df, parameter_col, outcome_col)
        evaluate_diagnostic_performance_categorical_bootstrap(df, parameter_col, outcome_col)

        

# Bootstrap for continuous features

In [ ]:
def bootstrap_diagnostic_performance(sub_df, parameter_col, outcome_col):

    if set_prevalence is None:
        prevalence = (sub_df[outcome_col]=='yes').sum() / sub_df.shape[0]
    else:
        prevalence = set_prevalence
        
    fpr, tpr, thresholds = roc_curve(y_true = sub_df[outcome_col], 
                                     y_score = sub_df[parameter_col],
                                     pos_label = 'yes')
    
    auroc = roc_auc_score(y_true = sub_df[outcome_col]=='yes', 
                         y_score = sub_df[parameter_col])

    # calculate other metrics
    sens = tpr
    spec = 1 - fpr
    ppv = (sens*prevalence) / ((sens*prevalence) + ((1-spec)*(1-prevalence)))
    npv = (spec*(1-prevalence)) / (((1-sens)*prevalence)+(spec*(1-prevalence)))
    pos_lr = sens / (1-spec)
    neg_lr = (1-sens) / spec
    
    # write results to dataframe
    out_df = pd.DataFrame({
        'prevalence': [prevalence]*len(thresholds),
        'threshold': thresholds,
        'sensitivity': sens,
        'specificity': spec,
        'PPV': ppv,
        'NPV': npv,
        'positive_likelihood_ratio': pos_lr,
        'negative_likelihood_ratio': neg_lr,
        'AUROC': [auroc]*len(thresholds)
    })

    return out_df

In [ ]:
iterations = 10000

for parameter_col in parameter_columns:
    print('\n')
    print(parameter_col)
    for outcome_col in outcome_columns:
        print(outcome_col)
        
        # bootstrap
        available_df = df.loc[df[parameter_col].notna() & \
                            df[outcome_col].notna()].copy().reset_index(drop=True)
        print(available_df.shape)
        if available_df.shape[0] == 0:
            pass

        results_df = pd.DataFrame({
            'prevalence': [],
            'threshold': [],
            'sensitivity': [],
            'specificity': [],
            'PPV': [],
            'NPV': [],
            'positive_likelihood_ratio': [],
            'negative_likelihood_ratio': [],
            'AUROC': [],
            'bootstrap_iteration': []
        })

        for i in tqdm(range(iterations)):
            sampled_indices = np.random.choice(available_df.index, 
                                               size=available_df.shape[0],
                                               replace=True)
            bootstrap_df = available_df.loc[sampled_indices].copy().reset_index(drop=True)
            out_df = bootstrap_diagnostic_performance(bootstrap_df, parameter_col, outcome_col)
            out_df['bootstrap_iteration'] = i
            results_df = pd.concat([results_df, out_df],
                                  axis=0,
                                  ignore_index=True)

        # 95% CI for AUROC
        auroc_df = results_df.drop_duplicates(subset=['bootstrap_iteration']).copy().reset_index(drop=True)
        values = auroc_df['AUROC'].to_list()
        lower_CI = np.quantile(values, 0.025)
        upper_CI = np.quantile(values, 0.975)
        print('AUROC 95% CI: ({0:.2f}, {1:.2f})'.format(lower_CI, upper_CI))

        for thres in map_col_to_selected_thresholds[parameter_col]:
            print('\n')
            print('threshold:', thres)
            print('prevalence:', results_df.loc[0, 'prevalence'])
            selected_df = results_df.loc[results_df['threshold'].round(decimals=2)==thres].copy().reset_index(drop=True)
            print(selected_df.shape)
            for col in ['sensitivity','specificity','positive_likelihood_ratio','negative_likelihood_ratio','PPV','NPV',]:
                values = selected_df[col].to_list()
                lower_CI = np.quantile(values, 0.025)
                upper_CI = np.quantile(values, 0.975)
                print('{0} 95% CI: ({1:.3f}, {2:.3f})'.format(col, lower_CI, upper_CI))